## Reference: https://github.com/WuJie1010/Facial-Expression-Recognition.Pytorch

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/Colab Notebooks/csc420_bacloud"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/csc420_bacloud


In [ ]:
# !python preprocess_fer2013.py

In [ ]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import transforms as transforms
import numpy as np
import os

from torch.autograd import Variable

# import argparse
import utils
# dataclass
from fer import FER2013
# model declarations
from models import *

In [ ]:
class Opt():
  def __init__(self):
    self.model = "Xception"
    self.dataset = "FER2013"
    self.bs = 128
    self.lr = .01
  
  def __str__(self):
    return f"model = {self.model}\ndataset = {self.dataset}\nbatchsize = {self.bs}\nlearning rate = {self.lr}"

opt = Opt()
print(opt)

model = Xception
dataset = FER2013
batchsize = 128
learning rate = 0.01


In [ ]:
# params
use_cuda = torch.cuda.is_available()
print("training on CPU" if not use_cuda else "GPU in place!")

best_validation_acc = 0  # best validation accuracy
best_validation_acc_epoch = 0
best_test_acc = 0  # best test accuracy
best_test_acc_epoch = 0

learning_rate_decay_start = 40  # 50
learning_rate_decay_every = 5 # 5
learning_rate_decay_rate = 0.9 # 0.9

cut_size = 44
total_epoch = 250

path = os.path.join(opt.dataset + '_' + opt.model)
print(f"savepath is {path}")

GPU in place!
savepath is FER2013_Xception


In [ ]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(cut_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.TenCrop(cut_size),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
])

trainset = FER2013(split = 'Training', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=opt.bs, shuffle=True, num_workers=1)
validationset = FER2013(split = 'validation', transform=transform_test)
validationloader = torch.utils.data.DataLoader(validationset, batch_size=opt.bs, shuffle=False, num_workers=1)
testset = FER2013(split = 'test', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=opt.bs, shuffle=False, num_workers=1)

==> Preparing data..


In [ ]:
# Model

if opt.model  == 'Resnet18':
    net = ResNet18()
elif opt.model == 'Xception':
    net = Xception(input_channel=3)
elif opt.model.startswith('VGG'):
    net = VGG(opt.model)

print(f'==> Building model {opt.model}..')

if use_cuda:
    net.cuda()

criterion = nn.CrossEntropyLoss()
# SGD could be more generalizable but it is slower in convergence
# weight decay option adds regularization. 
optimizer = optim.SGD(net.parameters(), lr=opt.lr, momentum=.9, weight_decay=5e-4)
# optimizer = optim.Adam(net.parameters(), lr=opt.lr, weight_decay=5e-4)

==> Building model Xception..


In [ ]:
# Training
def train(epoch):
    print('\n---\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    if epoch > learning_rate_decay_start and learning_rate_decay_start >= 0:
        frac = (epoch - learning_rate_decay_start) // learning_rate_decay_every
        decay_factor = learning_rate_decay_rate ** frac
        current_lr = opt.lr * decay_factor
        utils.set_lr(optimizer, current_lr)  # set the decayed rate
    else:
        current_lr = opt.lr
    print('learning_rate: %s' % str(current_lr))

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        utils.clip_gradient(optimizer, 0.1)
        optimizer.step()
        train_loss += loss.data
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        # for command line
        # utils.progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
        #     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

    Train_acc = 100.*correct/total
    print(f"Trainning Accuracy: {Train_acc:.10f}")

In [ ]:
def validation(epoch):
    global validation_acc
    global best_validation_acc
    global best_validation_acc_epoch
    net.eval()
    validation_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(validationloader):
        bs, ncrops, c, h, w = np.shape(inputs)
        inputs = inputs.view(-1, c, h, w)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
          # inputs, targets = Variable(inputs, volatile=True), Variable(targets)
          outputs = net(inputs)
          outputs_avg = outputs.view(bs, ncrops, -1).mean(1)  # avg over crops
          loss = criterion(outputs_avg, targets)
          validation_loss += loss.data
          _, predicted = torch.max(outputs_avg.data, 1)
          total += targets.size(0)
          correct += predicted.eq(targets.data).cpu().sum()

          # for command line
          # utils.progress_bar(batch_idx, len(validationloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
          #                   % (validation_loss / (batch_idx + 1), 100. * correct / total, correct, total))

    # Save checkpoint.
    validation_acc = 100.*correct/total
    print("validation_acc: %0.3f" % validation_acc)
    if validation_acc > best_validation_acc:
        print('Saving..')
        print("best_validation_acc: %0.3f" % validation_acc)
        
        state = {
            'net': net.state_dict() if use_cuda else net,
            'acc': validation_acc,
            'epoch': epoch,
        }
        if not os.path.isdir(path):
            os.mkdir(path)
        torch.save(state, os.path.join(path,'validation_model.t7'))
        best_validation_acc = validation_acc
        best_validation_acc_epoch = epoch

In [ ]:
# deprecated, we will be keeping test set only for the best validation model
def test(epoch):
    global test_acc
    global best_test_acc
    global best_test_acc_epoch
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        bs, ncrops, c, h, w = np.shape(inputs)
        inputs = inputs.view(-1, c, h, w)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
          # inputs, targets = Variable(inputs, volatile=True), Variable(targets)
          outputs = net(inputs)
          outputs_avg = outputs.view(bs, ncrops, -1).mean(1)  # avg over crops
          loss = criterion(outputs_avg, targets)
          test_loss += loss.data
          _, predicted = torch.max(outputs_avg.data, 1)
          total += targets.size(0)
          correct += predicted.eq(targets.data).cpu().sum()

          utils.progress_bar(batch_idx, len(validationloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
              % (test_loss / (batch_idx + 1), 100. * correct / total, correct, total))
    # Save checkpoint.
    test_acc = 100.*correct/total

    if test_acc > best_test_acc:
        print('Saving..')
        print("best_test_acc: %0.3f" % max(test_acc, best_test_acc))
        state = {
            'net': net.state_dict() if use_cuda else net,
            'best_validation_acc': best_validation_acc,
            'best_test_acc': test_acc,
            'best_validation_acc_epoch': best_validation_acc_epoch,
            'best_test_acc_epoch': epoch,
        }
        if not os.path.isdir(path):
            os.mkdir(path)
        torch.save(state, os.path.join(path,'test_model.t7'))
        best_test_acc = test_acc
        best_test_acc_epoch = epoch

In [ ]:
print(opt)

# training loop
for epoch in range(total_epoch):
    train(epoch)
    validation(epoch)
    # test(epoch)

print("best_validation_acc: %0.3f" % best_validation_acc)
print("best_validation_acc_epoch: %d" % best_validation_acc_epoch)
# print("best_test_acc: %0.3f" % best_test_acc)
# print("best_test_acc_epoch: %d" % best_test_acc_epoch)


---
Epoch: 0
learning_rate: 0.01
Trainning Accuracy: 34.5989074707
validation_acc: 32.042
Saving..
best_validation_acc: 32.042

---
Epoch: 1
learning_rate: 0.01
Trainning Accuracy: 46.4592971802
validation_acc: 36.890
Saving..
best_validation_acc: 36.890

---
Epoch: 2
learning_rate: 0.01
Trainning Accuracy: 51.5099792480
validation_acc: 41.906
Saving..
best_validation_acc: 41.906

---
Epoch: 3
learning_rate: 0.01
Trainning Accuracy: 54.8573608398
validation_acc: 41.098

---
Epoch: 4
learning_rate: 0.01
Trainning Accuracy: 57.0378608704
validation_acc: 43.689
Saving..
best_validation_acc: 43.689

---
Epoch: 5
learning_rate: 0.01
Trainning Accuracy: 58.2918243408
validation_acc: 38.339

---
Epoch: 6
learning_rate: 0.01
Trainning Accuracy: 59.8383789062
validation_acc: 37.838

---
Epoch: 7
learning_rate: 0.01
Trainning Accuracy: 61.1445884705
validation_acc: 27.974

---
Epoch: 8
learning_rate: 0.01
Trainning Accuracy: 62.2766380310
validation_acc: 43.633

---
Epoch: 9
learning_rate: 0.01

In [ ]:
print(f"evaluating for dataset_model = {path}")

# load the best model according to validation
net.load_state_dict(torch.load(path + "/validation_model.t7")["net"])
net.eval()

test_loss = 0
correct = 0
total = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
    bs, ncrops, c, h, w = np.shape(inputs)
    inputs = inputs.view(-1, c, h, w)
    if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    with torch.no_grad():
        # inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        outputs = net(inputs)
        outputs_avg = outputs.view(bs, ncrops, -1).mean(1)  # avg over crops
        loss = criterion(outputs_avg, targets)
        test_loss += loss.data
        _, predicted = torch.max(outputs_avg.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
  
print(f"Best model chosen based on best validation accuracy, ")
print(f"Achieved test accuracy {100. * correct / total}")

evaluating for dataset_model = FER2013_Xception
Best model chosen based on best validation accuracy, 
Achieved test accuracy 56.95179748535156


In [ ]:
max(1, 2)

2